In [1]:
!pip install transformers
import json 
import numpy as np
import tensorflow as tf
import re
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel,BertConfig
import os
import string 

configuration = BertConfig()  # default parameters and configuration for BERT
max_len = 384

     |████████████████████████████████| 1.3MB 3.3MB/s 
     |████████████████████████████████| 1.1MB 19.4MB/s 
     |████████████████████████████████| 890kB 32.6MB/s 
     |████████████████████████████████| 2.9MB 35.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=d2f2fe005939cf0a609d81f60464e2ca320da655bbd4c75923a498de8830649b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


## Bert Tokenizer and model for fine-tuning

In [2]:
slow_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
save_path = "bert_base_uncased/"
if not os.path.exists(save_path):
    os.makedirs(save_path)
slow_tokenizer.save_pretrained(save_path)

# Load the fast tokenizer from saved file
tokenizer = BertWordPieceTokenizer("bert_base_uncased/vocab.txt", lowercase=True)


## Preprocessing of data
- Create each example as SQuAD class object

In [3]:
class SQuADEntry:
    def __init__(self, context, question, basic_answer, more_answers, start_idx):
        self.context = context
        self.question = question
        self.basic_answer = basic_answer
        self.more_answers = more_answers
        self.start_idx = start_idx
        self.end_idx = None
        self.start_idx_token = start_idx
        self.end_idx_token = None
        self.offsets = None
        self.input_ids = None 
        self.attention_mask = None
        self.token_type_ids = None
        self.validExample= True
    def __call__(self):

         # Clean context, answer and question
        self.context = " ".join(str(self.context).split())
        self.question = " ".join(str(self.question).split())
        self.basic_answer = " ".join(str(self.basic_answer).split())


        #Calculate end_idx
        self.end_idx = self.start_idx + len(self.basic_answer)
        if (self.end_idx >=len(self.context)):
            self.validExample= False
            return

        #find characters of context that are part of answer
        is_part_of_answer = [0]*len(self.context)
        for i in range (self.start_idx, self.end_idx):
            is_part_of_answer[i] = 1

        contextTokenizer =  tokenizer.encode(self.context)  
        #find index of token that corresponds to start and the end of the answer
        answer_id_token=[]
        for idx, (start,end) in enumerate(contextTokenizer.offsets):
            if (sum(is_part_of_answer[start:end]) >0 ):
                answer_id_token.append(idx)
        #data to predict
        if len(answer_id_token) == 0 :
            self.validExample=False
            return         
        self.start_idx_token = answer_id_token[0]
        self.end_idx_token = answer_id_token[-1]
        self.offsets = contextTokenizer.offsets
        
        # work on question
        questionTokinizer  = tokenizer.encode(self.question)

        #Create model's inputs 
        self.input_ids = contextTokenizer.ids + questionTokinizer.ids[1:]
        # Dropout long examples
        if (max_len - len(self.input_ids) < 0 ):
            self.validExample= False
            return
        self.attention_mask = [1] * len (self.input_ids)
        self.token_type_ids = [0] * len(contextTokenizer.ids) + \
                                [1]*len(questionTokinizer.ids[1:])

                                




## Load data from JSON

In [4]:
def create_squad_examples(raw_data):
    squad_examples = []
    for item in raw_data["data"]:
        for para in item["paragraphs"]:
            context = para["context"]
            for qa in para["qas"]:
                question = qa["question"]
                answer_text = qa["answers"][0]["text"]
                all_answers = [_["text"] for _ in qa["answers"]]
                start_char_idx = qa["answers"][0]["answer_start"]

                #context, question, basic_answer, more_answers, start_idx
                squad_eg = SQuADEntry(
                 context,question, answer_text, all_answers, start_char_idx,
                )
                squad_eg()
                squad_examples.append(squad_eg)
    return squad_examples

# Data url 
train_data_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json"
train_path = tf.keras.utils.get_file("train.json", train_data_url)
eval_data_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json"
eval_path = tf.keras.utils.get_file("eval.json", eval_data_url)  
#open JSON's
with open(train_path) as f:
    raw_train_data = json.load(f)

with open(eval_path) as f:
    raw_eval_data = json.load(f)


4857856/4854279 [==============================] - 0s 0us/step


## Creating training examples

In [5]:
def createTrainingExamples(sqaud_examples):
    dict_training={
        "input_ids" : [],
        "attention_mask" : [],
        "token_type_ids" : [],
        "start_idx_token" : [],
        "end_idx_token" : []
    }
    count = 0
    for x in sqaud_examples:
        if (x.validExample):
            dict_training["input_ids"].append(x.input_ids)
            dict_training["attention_mask"].append(x.attention_mask)
            dict_training["token_type_ids"].append(x.token_type_ids)
            dict_training['start_idx_token'].append(x.start_idx_token)
            dict_training['end_idx_token'].append(x.end_idx_token)
        else:
            count+=1
    print(count)            
    #convert to numpy arrays 
    for key in dict_training:
        dict_training[key]= np.array(dict_training[key])
        print(dict_training[key])
    #padding
    dict_training["input_ids"] = tf.keras.preprocessing.sequence.pad_sequences( dict_training["input_ids"],padding='post',maxlen=max_len)
    dict_training["attention_mask"]=tf.keras.preprocessing.sequence.pad_sequences(dict_training["attention_mask"],padding='post',maxlen=max_len)
    dict_training["token_type_ids"]=tf.keras.preprocessing.sequence.pad_sequences(dict_training["token_type_ids"],padding='post',maxlen=max_len)
            
    X= [dict_training["input_ids"], dict_training["attention_mask"], dict_training["token_type_ids"]]
    Y= [dict_training["start_idx_token"], dict_training["end_idx_token"]]
    return X,Y    




In [6]:
sQUADTrain  = create_squad_examples(raw_train_data)
XTrain, YTrain = createTrainingExamples(sQUADTrain)
sQUADEval  = create_squad_examples(raw_eval_data)
XEval, YEval = createTrainingExamples(sQUADEval)

1463
[list([101, 6549, 2135, 1010, 1996, 2082, 2038, 1037, 3234, 2839, 1012, 10234, 1996, 2364, 2311, 1005, 1055, 2751, 8514, 2003, 1037, 3585, 6231, 1997, 1996, 6261, 2984, 1012, 3202, 1999, 2392, 1997, 1996, 2364, 2311, 1998, 5307, 2009, 1010, 2003, 1037, 6967, 6231, 1997, 4828, 2007, 2608, 2039, 14995, 6924, 2007, 1996, 5722, 1000, 2310, 3490, 2618, 4748, 2033, 18168, 5267, 1000, 1012, 2279, 2000, 1996, 2364, 2311, 2003, 1996, 13546, 1997, 1996, 6730, 2540, 1012, 3202, 2369, 1996, 13546, 2003, 1996, 24665, 23052, 1010, 1037, 14042, 2173, 1997, 7083, 1998, 9185, 1012, 2009, 2003, 1037, 15059, 1997, 1996, 24665, 23052, 2012, 10223, 26371, 1010, 2605, 2073, 1996, 6261, 2984, 22353, 2135, 2596, 2000, 3002, 16595, 9648, 4674, 2061, 12083, 9711, 2271, 1999, 8517, 1012, 2012, 1996, 2203, 1997, 1996, 2364, 3298, 1006, 1998, 1999, 1037, 3622, 2240, 2008, 8539, 2083, 1017, 11342, 1998, 1996, 2751, 8514, 1007, 1010, 2003, 1037, 3722, 1010, 2715, 2962, 6231, 1997, 2984, 1012, 102, 2000, 3183, 2

In [8]:
print(f"{len(sQUADTrain)} training points created.")
print(f"{len(sQUADEval)} evaluation points created.")


87599 training points created.
10570 evaluation points created.


## Create model


In [9]:
def createModel():
    #Bert model
    TFmodel = TFBertModel.from_pretrained('bert-base-uncased')
    #Create model's inputs
    inputIds= tf.keras.layers.Input((max_len,), dtype=tf.int32)
    inputAttention= tf.keras.layers.Input((max_len,), dtype=tf.int32)
    inputTokenType= tf.keras.layers.Input((max_len,), dtype=tf.int32)
    
    #Pass through BERT model
    bertOut= TFmodel(inputIds,attention_mask=inputAttention,token_type_ids=inputTokenType)[0]

    #Create classification layer
    start_logits= tf.keras.layers.Dense(1, use_bias= False)(bertOut)
    start_logits= tf.keras.layers.Flatten()(start_logits)    
    end_logits= tf.keras.layers.Dense(1, use_bias=False)(bertOut)
    end_logits= tf.keras.layers.Flatten()(end_logits)
    #Pass though softmax
    start_prob = tf.keras.layers.Activation(tf.keras.activations.softmax)(start_logits)
    end_prob = tf.keras.layers.Activation(tf.keras.activations.softmax)(end_logits)

    model =tf.keras.Model(inputs=[ inputIds, inputAttention, inputTokenType],
                          outputs=[start_prob, end_prob])
    loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

    optimizer = tf.keras.optimizers.Adam(lr= 5e-5)
    model.compile(optimizer = optimizer, loss= [loss, loss])
    return model





## Run on TPU's

In [10]:
use_tpu = True
if use_tpu:
    # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model = createModel()
else:
    model = createModel()

model.summary()

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.30.125.34:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.30.125.34:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 384)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 384)]        0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 384)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 384, 768), ( 109482240   input_1[0][0]                    
                                                                 input_2[0][0]         

## Train and evaluate model on validation data

In [11]:
def normalizeText(text):
    # lower case
    text = text.lower()
    #remove a, an, the 
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    text = re.sub(regex, " ", text)

    #remove puctuation
    puctuation = set(string.punctuation)
    text = "".join(_ for _ in text if _ not in puctuation)
    #remove extra white spaces
    text = " ".join(text.split())
    return text

class ExactMatchCallback(tf.keras.callbacks.Callback):
    def __init__(self, x_eval, y_eval):
        self.x_eval=x_eval
        self.y_eval=y_eval

    def on_epoch_end(self,epoch,logs=None):
        pred_start, pred_end = self.model.predict(self.x_eval)
        valid_examples = [_ for _ in sQUADEval if _.validExample]
        counter = 0
        for idx, (start,end) in enumerate(zip(pred_start, pred_end)):
            squadV = valid_examples[idx]
            #start and end is max_len-demensionality vector.
            token_idx_start = np.argmax(start)
            token_idx_end = np.argmax(end)
            if (token_idx_start >= len(squadV.offsets)):
                continue
            # start and end indices in context
            start_char_idx = squadV.offsets[token_idx_start][0]
    
            #predicted answer
            if (token_idx_end < len(squadV.offsets)):
                end_char_idx = squadV.offsets[token_idx_end][1]
                predictedAnswer = squadV.context[start_char_idx:end_char_idx]
            else:
                predictedAnswer = squadV.context[start_char_idx:]
            #normalize texts to check similarity with the other available answers
            available_answers = [normalizeText(_) for _ in squadV.more_answers]
            predictedAnswer = normalizeText(predictedAnswer)
            #Check for similarity of answers
            if (predictedAnswer in available_answers):
                counter+=1
        acc = counter/len(self.y_eval[0])
        print()
        print("Exact Match: ", acc)    



In [12]:
model.fit(x=XTrain, y=YTrain, batch_size= 64, epochs=3, verbose=1, callbacks=[ExactMatchCallback(XEval,YEval)])


Epoch 1/3
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


   2/1346 [..............................] - ETA: 2:37 - loss: 11.7489 - activation_4_loss: 6.0357 - activation_5_loss: 5.7132WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0138s vs `on_train_batch_end` time: 0.2145s). Check your callbacks.


1346/1346 [==============================] - ETA: 0s - loss: 2.5607 - activation_4_loss: 1.3351 - activation_5_loss: 1.2255
Exact Match:  0.7790146162036589
1346/1346 [==============================] - 363s 270ms/step - loss: 2.5607 - activation_4_loss: 1.3351 - activation_5_loss: 1.2255
Epoch 2/3
1346/1346 [==============================] - ETA: 0s - loss: 1.5583 - activation_4_loss: 0.8285 - activation_5_loss: 0.7298
Exact Match:  0.7906301422901946
1346/1346 [==============================] - 315s 234ms/step - loss: 1.5583 - activation_4_loss: 0.8285 - activation_5_loss: 0.7298
Epoch 3/3
1346/1346 [==============================] - ETA: 0s - loss: 1.1026 - activation_4_loss: 0.5924 - activation_5_loss: 0.5102
Exact Match:  0.784338398993321
1346/1346 [==============================] - 315s 234ms/step - loss: 1.1026 - activation_4_loss: 0.5924 - activation_5_loss: 0.5102
